# Dataframe cleaning

In [14]:
import pandas as pd
data_frame = pd.read_csv("googleplaystore.csv")
data_frame.sample(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
4439,Band O'Clock,LIFESTYLE,3.8,1140,746k,"100,000+",Free,0,Everyone,Lifestyle,"December 3, 2011",0.11,2.1 and up
10754,SCM FPS Status,BUSINESS,4.2,123,3.3M,"10,000+",Free,0,Everyone,Business,"April 28, 2018",3.5,2.3 and up
1714,Wordscapes,GAME,4.8,230710,87M,"10,000,000+",Free,0,Everyone,Word,"August 2, 2018",1.0.47,4.1 and up


Which of the following column(s) has/have null values?

In [15]:
data_frame.isna().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

Clean the `Rating` column and the other columns containing null values and greater than 5

In [16]:
data_frame = data_frame.loc[data_frame["Rating"] <= 5]
(data_frame["Rating"].isna().sum() & (data_frame["Rating"] > 5).sum())

0

Convert a column to numeric pd.to_...

In [17]:
data_frame["Reviews"] = pd.to_numeric(data_frame["Reviews"])

How many duplicated apps are there?

In [18]:
data_frame.loc[
  (data_frame.duplicated(subset=["App"], keep=False)) &
  (~data_frame.duplicated(keep=False)) # Duplicated by App column but different in other columns
].sort_values(by="App", ascending=True)


(data_frame.duplicated(subset=["App"], keep=False)).value_counts()

False    7407
True     1959
Name: count, dtype: int64

Drop duplicated apps kepping the ones with the greatest number of reviews

In [19]:
# subset: column
# keep: determine the column to keep
# inplace: if true modify de data frame instead of creating a new one
data_frame.drop_duplicates(subset=["App"], keep="last", inplace=True) 

Modify the Category column by replacing "_" with white spaces and capitalize it

In [20]:
data_frame["Category"] = data_frame["Category"].str.replace("_", " ")
data_frame["Category"] = data_frame["Category"].str.capitalize()

Clean and convert the `Installs` column by replacing '+', ',' and convert it to numeric

In [23]:
data_frame["Installs"] = data_frame["Installs"].str.replace(",", "").str.replace("+", "")
data_frame["Installs"] = pd.to_numeric(data_frame["Installs"])

<class 'pandas.core.frame.DataFrame'>
Index: 8196 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             8196 non-null   object 
 1   Category        8196 non-null   object 
 2   Rating          8196 non-null   float64
 3   Reviews         8196 non-null   int64  
 4   Size            8196 non-null   object 
 5   Installs        8196 non-null   int64  
 6   Type            8196 non-null   object 
 7   Price           8196 non-null   object 
 8   Content Rating  8196 non-null   object 
 9   Genres          8196 non-null   object 
 10  Last Updated    8196 non-null   object 
 11  Current Ver     8192 non-null   object 
 12  Android Ver     8194 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 896.4+ KB


Clean the `Size` column by replacing 'k', 'M' and 'Varies with device' and convert it to numeric

In [72]:

data_frame.loc[data_frame["Size"] == "Varies with device", "Size"] = "0"

data_frame.loc[data_frame["Size"].str.contains("k"), "Size"] = (
  pd.to_numeric(
  data_frame.loc[data_frame["Size"].str.contains("k"), "Size"].str.replace("k", "")
) * 1024).astype(str)

data_frame.loc[data_frame["Size"].str.contains("M"), "Size"] = (
  pd.to_numeric(
  data_frame.loc[data_frame["Size"].str.contains("M"), "Size"].str.replace("M", "")
) * (1024 * 1024)).astype(str)

<class 'pandas.core.frame.DataFrame'>
Index: 8196 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             8196 non-null   object 
 1   Category        8196 non-null   object 
 2   Rating          8196 non-null   float64
 3   Reviews         8196 non-null   int64  
 4   Size            8196 non-null   float64
 5   Installs        8196 non-null   int64  
 6   Type            8196 non-null   object 
 7   Price           8196 non-null   object 
 8   Content Rating  8196 non-null   object 
 9   Genres          8196 non-null   object 
 10  Last Updated    8196 non-null   object 
 11  Current Ver     8192 non-null   object 
 12  Android Ver     8194 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 896.4+ KB


Clean the `Price` column by replacing the '$' character

In [79]:
data_frame["Price"] = data_frame["Price"].str.replace("$", "")
data_frame["Price"] = pd.to_numeric(data_frame["Price"])

Create a column called `Distribution` depending if the app is free or not

In [91]:
# My way
data_frame["Distribution"] = "Free"
data_frame.loc[data_frame["Price"] != 0, "Distribution"] = "Paid"

# Good way
data_frame["Distribution"] = data_frame["Price"].apply(lambda p: "Free" if p > 0 else "Paid")

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Distribution
290,TurboScan: scan documents and receipts in PDF,Business,4.7,11442,7130316.8,100000,Paid,4.99,Everyone,Business,"March 25, 2018",1.5.2,4.0 and up,Paid
291,Tiny Scanner Pro: PDF Doc Scan,Business,4.8,10295,40894464.0,100000,Paid,4.99,Everyone,Business,"April 11, 2017",3.4.6,3.0 and up,Paid
427,Puffin Browser Pro,Communication,4.0,18247,0.0,100000,Paid,3.99,Everyone,Communication,"July 5, 2018",7.5.3.20547,4.1 and up,Paid
481,AMBW Dating App: Asian Men Black Women Interra...,Dating,3.5,2,17825792.0,100,Paid,7.99,Mature 17+,Dating,"January 21, 2017",1.0.1,4.0 and up,Paid
571,"Moco+ - Chat, Meet People",Dating,4.2,1546,0.0,10000,Paid,3.99,Mature 17+,Dating,"June 19, 2018",2.6.139,4.1 and up,Paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10690,FO Bixby,Personalization,5.0,5,881664.0,100,Paid,0.99,Everyone,Personalization,"April 25, 2018",0.2,7.0 and up,Paid
10697,Mu.F.O.,Game,5.0,2,16777216.0,1,Paid,0.99,Everyone,Arcade,"March 3, 2017",1.0,2.3 and up,Paid
10760,Fast Tract Diet,Health and fitness,4.4,35,2516582.4,1000,Paid,7.99,Everyone,Health & Fitness,"August 8, 2018",1.9.3,4.2 and up,Paid
10782,Trine 2: Complete Story,Game,3.8,252,11534336.0,10000,Paid,16.99,Teen,Action,"February 27, 2015",2.22,5.0 and up,Paid


# Analysis

What app has the most reviews?

In [96]:
data_frame.sort_values(by="Reviews", ascending=False).head(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Distribution
3943,Facebook,Social,4.1,78128208,0.0,1000000000,Free,0.0,Teen,Social,"August 3, 2018",Varies with device,Varies with device,Free


Wich is the category with the most uplodaded apps?

In [100]:
data_frame.value_counts(subset="Category")

Category
Family                 1681
Game                    879
Tools                   720
Finance                 302
Lifestyle               301
Productivity            301
Personalization         298
Medical                 291
Photography             263
Business                262
Sports                  262
Communication           257
Health and fitness      243
News and magazines      204
Social                  203
Travel and local        187
Shopping                180
Books and reference     169
Video players           148
Dating                  133
Maps and navigation     118
Education               101
Food and drink           94
Entertainment            79
Auto and vehicles        73
Weather                  72
Libraries and demo       64
House and home           61
Art and design           59
Comics                   54
Parenting                50
Events                   45
Beauty                   42
Name: count, dtype: int64

To wich category belongs the most expansive app?

In [102]:
data_frame.sort_values(by="Price", ascending=False).head(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Distribution
4367,I'm Rich - Trump Edition,Lifestyle,3.6,275,7654604.8,10000,Paid,400.0,Everyone,Lifestyle,"May 3, 2018",1.0.1,4.1 and up,Paid


What is the name of the most expensive game?

In [106]:
data_frame.loc[data_frame["Category"] == "Game"].sort_values(by="Price", ascending=False).head(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Distribution
4203,The World Ends With You,Game,4.6,4108,13631488.0,10000,Paid,17.99,Everyone 10+,Arcade,"December 14, 2015",1.0.4,4.0 and up,Paid
